### Redes de classificação
Selecione um dataset e crie uma rede de classificação. Avalie o resultado com duas configurações diferentes. As configurações devem mudar no mínimo 3 hiperparâmetros (por exemplo, parâmetros do optimizer,  loss, número de camadas, função de ativação, normalização).

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torchvision.models import mobilenet_v3_small, MobileNet_V3_Small_Weights
from torch.utils.data import DataLoader

In [18]:
weights = MobileNet_V3_Small_Weights.DEFAULT

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=weights.transforms())
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=weights.transforms())

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)

Files already downloaded and verified
Files already downloaded and verified


In [19]:
model = mobilenet_v3_small(pretrained=True)
print(model.classifier)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, 10)
print(model.classifier)

Sequential(
  (0): Linear(in_features=576, out_features=1024, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1024, out_features=1000, bias=True)
)
Sequential(
  (0): Linear(in_features=576, out_features=1024, bias=True)
  (1): Hardswish()
  (2): Dropout(p=0.2, inplace=True)
  (3): Linear(in_features=1024, out_features=10, bias=True)
)


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
model = model.to(device)

Device: cpu


In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0)

In [22]:
def train(model, train_loader, criterion, optimizer, device, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()  # Zerar os gradientes
            outputs = model(inputs)  # Passar as entradas pela rede
            loss = criterion(outputs, labels)  # Calcular a perda
            loss.backward()  # Calcular os gradientes
            optimizer.step()  # Atualizar os pesos

            # Estatísticas de treinamento
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct/total:.2f}%")

In [23]:
def evaluate(model, test_loader, device):
    model.eval()  # Colocar o modelo em modo de avaliação
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

In [24]:
train(model, train_dataloader, criterion, optimizer, device, num_epochs=10)
evaluate(model, test_dataloader, device)

KeyboardInterrupt: 